<a href="https://colab.research.google.com/github/joshlevin91/Benefit-Corp-Web-Scraper/blob/master/BCorporationWebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Selenium and Chromedriver**

In [80]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/

**Get benefit corporation information**

In [81]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# Get company website
def getWebsite(url):
  source = requests.get(url).text
  soup = BeautifulSoup(source, 'lxml')
  res = soup.find_all('div',{'class':'opacity-60'})
  for r in res:
    a = r.find('a', href=True)
    if a:
      return a['href']

# Get region string
def getRegStr(region):
  regstr = region[0]
  if regstr == "Netherlands":
    return "The Netherlands"
  if len(region) == 2:
      regstr += " " + region[1]
  return regstr

# Get bcorporation URL by region and page
def getURL(region, page):
  url = 'https://www.bcorporation.net/en-us/find-a-b-corp/search?page='
  url += str(page)
  url += '&sortBy=companies-production-en-us-latest-certification-desc&refinement=countries%3D'
  url += region[0]
  if len(region) == 2:
    url += '%20' + region[1]
  return url

# Get company info at a URL
def getCompanyInfo(region, url):
  driver.get(url)
  time.sleep(0.3) # Make sure page loads
  html = driver.page_source
  soup = BeautifulSoup(html, "html.parser")

  # Iterate through each company on page
  info = []
  for company in soup.find_all("li", {"class":"ais-Hits-item"}):
    name = company.find("div", {"data-testid":"company-name"}).text
    description = company.find("p").text
    cert = company.find("div", {"class":"flex-grow text-gray-dark"}).text.lstrip('Certified since: ')

    # Go to company page to get its website
    a = company.find('a', href=True)
    bcorp_company_page = 'https://www.bcorporation.net' + a['href']
    website = getWebsite(bcorp_company_page)

    company_info = [region, name, website, description, cert]
    info.append(company_info)

  return info

# Get info on corporations located in specified regions (sorted by newest)
def getCorporationInfo(regions):
  corporations = []

  # Iterate through regions
  for region in regions:
    regstr = getRegStr(region)
    print("Collecting data from " + regstr + "...")
    
    # Iterate through pages
    page = 1
    while True:
      url = getURL(region, page)
      info = getCompanyInfo(regstr, url)

      # Last page reached when no more info is available
      if not info:
        print()
        break
      else:
        print("Page " + str(page))
        corporations.extend(info)
        page += 1

  return corporations

# Initiate the webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

# Get corporation info
regions = [['United', 'States'], ['United', 'Kingdom'], ['Canada'], ['Australia'], ['France'], ['Netherlands', 'The'], ['Italy'], ['Spain'], ['Germany']]
corporations = getCorporationInfo(regions)

# Close the webdriver
driver.close()

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
Page 66
Page 67
Page 68
Page 69
Page 70
Page 71
Page 72
Page 73
Page 74
Page 75
Page 76
Page 77
Page 78
Page 79
Page 80
Page 81
Page 82
Page 83
Page 84
Page 85
Page 86
Page 87
Page 88
Page 89
Page 90
Page 91
Page 92
Page 93
Page 94
Page 95
Page 96
Page 97
Page 98
Page 99
Page 100

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27


**Write benefit corporation information to a Google worksheet**

In [82]:
import time

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Create and open worksheet
gc.create('B Corps')
worksheet = gc.open('B Corps').sheet1

# Add corporation data to worksheet
worksheet.insert_row(['Region', 'Name', 'Website', 'Description', 'Certification Date'])
worksheet.format('A1:E1', {'textFormat': {'bold': True}})
for c in corporations:
  time.sleep(0.1) #  Google Sheets API has a limit on the request rate
  worksheet.append_row(c)

**Go to https://sheets.google.com to see your new spreadsheet.**